# PCA with Numpy
In this notebook, we use Numpy to study Principal Component Analysis (PCA), a technique that allows to project data onto a lower-dimensional space without losing too much information.

We import the following packages:
- Numpy for linear algebra tools
- Matplotlib for visualization
- Pandas to manipulate datasets.

Pandas is described more extensively in another notebook.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

We now import some specific modules from Pandas and Numpy

In [ ]:
from pandas import Series, DataFrame
from numpy import linalg as LA

Next, we also import modules to provide pretty-printing of matrices and vectors.

In [ ]:
from sympy import init_printing, Matrix, symbols, Rational
import sympy as sym
from warnings import filterwarnings
init_printing(use_latex = 'mathjax')
filterwarnings('ignore')

We now display and then load a dataset for the classification of wine quality. This uses some Pandas functions that are described in a different notebook.

In [ ]:
!cat 'Datasets/Wine.csv'

In [ ]:
dataset = pd.read_csv("Datasets/Wine.csv")
dataset.info()

The dataset has 178 points, each represented as a 14-dimensional vector.

In [ ]:
dataset.shape

Labels are stored in the `Customer_Segment` column. We print the distinct labels by selecting the column named `Customer_Segment` and using the `unique` function of Numpy.

In [ ]:
np.unique(dataset['Customer_Segment'])

Next, we separate datapoints from labels using the `iloc` method of Pandas to slice a matrix columnwise.

In [ ]:
Xh = dataset.iloc[:, 0:13].values # select submatrix only including columns 0-12
y = dataset.iloc[:, 13].values # select only column 13
X = Xh.T # transpose, now points are indexed by columns and coordinates by rows

Print dimensions and rank.

In [ ]:
X.shape, LA.matrix_rank(X)

Slicing in two dimensions

In [ ]:
Matrix(X)

In [ ]:
Matrix(X[:,0]) # first column of X

In [ ]:
Matrix(X[0,:]) # first row of X

In [ ]:
Matrix(X[:,:2]) # first two columns of X

We now compute the singular value decomposition (SVD): $$X = U\Sigma V^{\top} = \sum_{i=1}^d \sigma_i u_i v_i^{\top}$$

SVD is the generalization to non-square matrices of the spectral decomposition.
* $U$ and $V$ are orthonormal matrices
* The columns of $U$ are the eigenvectors of $XX^{\top}$
* The columns of $V$ are the eigenvectors of $X^{\top}X$
* The singular values (diagonal elements of $\Sigma$) are the square roots of the eigenvalues of $XX^{\top}$ or, equivalently, of $X^{\top}X$
* Hence, in particular, $XX^{\top}u_i = \sigma_i^2u_i$ and $X^{\top}Xv_i = \sigma_i^2v_i$ for all $i=1,\ldots,d$.

In [ ]:
U, s, Vh = LA.svd(X, full_matrices=False)
U.shape, s.shape, Vh.shape

In [ ]:
Matrix(np.diag(np.round(s, decimals=2)))

Let's check that, indeed, $XX^{\top}u_1 = \sigma_1^2 u_1$

In [ ]:
Matrix(X @ X.T @ U[:,0])

In [ ]:
Matrix(s[0]*s[0]*U[:,0])

We now take the two *principal eigenvectors*. Namely, the ones associated with the largest singular values.

In [ ]:
P = U[:,:2] # the first two columns of U
Matrix(P)

The linear operator $T_{P^{\top}} : \mathbb{R}^{13} \to \mathbb{R}^2$ corresponding to $P^{\top}$ projects the original data onto the subspace spanned by the two principal eigenvectors.

We thus project the columns of $X$ onto this space. 

In [ ]:
R = P.T @ X
Matrix(R)

We now plot the data points in this space (using color-coding for the labels).

In [ ]:
plt.scatter(R[0,:], R[1,:], c=y) # recall that y is the ndarray containing the labels

The principal eigenvectors correspond to orthogonal directions along which the variance is maximized.

In contrast, here is a projection that uses two eigenvectors associated to small singular values (note the change of scale in the axis).

In [ ]:
P = U[:,7:9]
R = P.T @ X
plt.scatter(R[0,:], R[1,:], c=y)

Intuitively, principal components help reduce the number of dimension without losing too much information in the data. This is formally stated as follows.

**Theorem (Eckhart-Young).** Let $X$ be a $d \times m$ matrix with SVD $U\Sigma V^{\top}$, where $\Sigma = \mathrm{diag}\big(\sigma_1,\dots,\sigma_r\big)$. Pick $1 \le k \le r$ and let
$
	X_k = U \Sigma_k V^{\top}
$
be the matrix such that $\Sigma_k = \mathrm{diag}\big(\sigma_1,\dots,\sigma_k,0\dots,0\big)$. Then
$$
	X_k = \underset{Z \,:\, \mathrm{rank}(Z) \le k}{\mathrm{argmin}} \|X - Z\|_F^2~.
$$

$\|\cdot\|_F$ is the Frobenius norm of a matrix, defined as
$$
\|X\|_F = \sqrt{\sum_{i,j}X_{i,j}^2}
$$
We now apply principal component analysis to a second dataset.

In [ ]:
dataset = pd.read_csv("Datasets/Iris.csv")
dataset.info()

In [ ]:
dataset.shape

Labels are stored in the column `Species`. However, unlike the previous example, they are encoded using strings.

In [ ]:
np.unique(dataset['Species'])

We then map these strings to integers. First, we create a dictionary, and then we invoke the method `map()` on the column `Species`

In [ ]:
label_to_int = {'Iris-setosa' : 1, 'Iris-versicolor' : 2, 'Iris-virginica' : 3}
dataset['Species'] = dataset['Species'].map(label_to_int)
np.unique(dataset['Species'])

In [ ]:
Xh = dataset.iloc[:,1:5].values
y = dataset.iloc[:,5].values
X = Xh.T

In [ ]:
X.shape, LA.matrix_rank(X)

In [ ]:
U, s, Vh = LA.svd(X, full_matrices=False)
U.shape, s.shape, Vh.shape

In [ ]:
Matrix(np.diag(np.round(s, decimals=2)))

In [ ]:
P = U[:,:2]
R = P.T @ X
plt.scatter(R[0,:], R[1,:], c=y)

Similarly to before, projecting onto components associated to small singular values causes the data to lump together.

In [ ]:
P = U[:,2:4]
R = P.T @ X
plt.scatter(R[0,:], R[1,:], c=y)